#### First download the station data for the Gulf States (TX, LA, AL, MS, FL)
#### Then clip the stations to the watershed counties 
#### Use the station names to build a URL and access the narrow results from the EPA Water Quality API

### URL Examples
##### EPA Water Quality Site:
##### https://www.waterqualitydata.us/

##### Download all the stations for USA states FL, AL, TX, MS, LA as a GEOJSON:
##### https://www.waterqualitydata.us/data/Station/search?countrycode=US&statecode=US%3A01&statecode=US%3A12&statecode=US%3A48&statecode=US%3A28&statecode=US%3A22&mimeType=geojsonv&zip=yes&providers=NWIS&providers=STEWARDS&providers=STORET

##### Download all the sample data for a given station (0800257-CC-05) as a CSV:
##### https://www.waterqualitydata.us/data/Result/search?siteid=0800257-CC-05&mimeType=csv&zip=yes&dataProfile=narrowResult&providers=NWIS&providers=STEWARDS&providers=STORET

In [1]:
import arcpy

In [ ]:
## coastal_watershed_counties item id = ff61c0c5154d4791ae51f00c4e4a3a98


GOMA_STATIONS = 
for x in GOMA_STATIONS:
